In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [46]:
def wrangle(n):
    df = pd.read_csv(n)

    mask_ba = df['place_with_parent_names'].str.contains("Capital Federal")
    mask_apt = df['property_type']== 'apartment'
    mask_price = df['price_aprox_usd'] < 400000
    df = df[mask_ba & mask_apt & mask_price]

    low, high = df['surface_covered_in_m2'].quantile([0.1, 0.9])
    mask_area = df['surface_covered_in_m2'].between(low, high)
    df= df[mask_area]

    df[['lat', 'lon']] = df['lat-lon'].str.split(',', expand=True)

    return df

In [47]:
df = wrangle('buenos-aires-real-estate-1.csv')

In [23]:
# xi = ['surface_covered_in_m2']
X = df[['surface_covered_in_m2']]
y = df['price_aprox_usd']

In [24]:
X.isnull().sum()

surface_covered_in_m2    0
dtype: int64

In [25]:
from sklearn.linear_model import LinearRegression

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [28]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [32]:
mean_y = y_train.mean()
y_pred_baseline = [mean_y] * len(y_train)

mae_baseline = mean_absolute_error(y_train, y_pred_baseline)
mse_baseline = mean_squared_error(y_train, y_pred_baseline)

In [34]:
print(f"The MAE baseline is {mae_baseline} and the MSE baseline is {mse_baseline}")

The MAE baseline is 44515.84888403399 and the MSE baseline is 3421488030.81776


In [36]:
model.score(X_train, y_train)

0.4720338803631867

In [38]:
prediction = model.predict(X_test)

In [39]:
model_mse = mean_squared_error(y_test, prediction)

In [40]:
model_mae = mean_absolute_error(y_test, prediction)

In [41]:
print(model_mse)

2143237958.885493


In [42]:
print(model_mae)

33926.462803777526


In [50]:
fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    width=600,
    height=600,
    hover_data=["price_aprox_usd"],
    color="price_aprox_usd",
)
fig.update_layout(mapbox_style="open-street-map")

fig.show()

TypeError: can only concatenate str (not "int") to str

In [48]:
df.head()

,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
4,sell,apartment,|Argentina|Capital Federal|Chacarita|,"-34.5846508988,-58.4546932614",129000.0,USD,1955949.6,129000.0,76.0,70.0,1697.368421,1842.857143,NaN,NaN,NaN,http://chacarita.properati.com.ar/10qlv_venta_...,-34.5846508988,-58.4546932614
9,sell,apartment,|Argentina|Capital Federal|Villa Luro|,"-34.6389789,-58.500115",87000.0,USD,1319128.8,87000.0,48.0,42.0,1812.500000,2071.428571,NaN,NaN,NaN,http://villa-luro.properati.com.ar/12m82_venta...,-34.6389789,-58.500115
29,sell,apartment,|Argentina|Capital Federal|Caballito|,"-34.615847,-58.459957",118000.0,USD,1789163.2,118000.0,NaN,54.0,NaN,2185.185185,NaN,2.0,NaN,http://caballito.properati.com.ar/11wqh_venta_...,-34.615847,-58.459957
40,sell,apartment,|Argentina|Capital Federal|Constitución|,"-34.6252219,-58.3823825",57000.0,USD,864256.8,57000.0,42.0,42.0,1357.142857,1357.142857,5.0,2.0,364.0,http://constitucion.properati.com.ar/k2f0_vent...,-34.6252219,-58.3823825
41,sell,apartment,|Argentina|Capital Federal|Once|,"-34.6106102,-58.4125107",90000.0,USD,1364616.0,90000.0,57.0,50.0,1578.947368,1800.000000,NaN,3.0,450.0,http://once.properati.com.ar/suwa_venta_depart...,-34.6106102,-58.4125107


In [ ]:
intercept = pipeline.named_steps[